In [1]:
import torch
from torch.utils.data import DataLoader
from torch.nn.functional import relu, avg_pool2d
import torch.nn as nn
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

import time
import random

# from GEM.gem import *
from GEM.args import *

from cifar import load_cifar10_data, split_into_classes, get_class_indexes 

# import quadprog

import os
import sys
sys.path.append(os.path.abspath("."))  # Adds the current directory

# CIFAR-10

In [2]:
# Path to the dataset
DATASET_PATH = 'cifar-10-python' 
CLASSES = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

train_data, train_labels, test_data, test_labels = load_cifar10_data(DATASET_PATH)
classes = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

# PRETRAINED MODEL

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from torchvision import transforms
import time
# make cuda available if available

from cifar import load_cifar10_data, show_image

# we want to create a resnet 18 model for 32x32 images
# avoid upscaling, the model will take 32x32 images on input as opposed to 224x224

initialisation = time.time()

class ResNet18CIFAR(torch.nn.Module):
    def __init__(self):
        super(ResNet18CIFAR, self).__init__()
        self.resnet = torch.hub.load('pytorch/vision:v0.10.0', 'resnet18', pretrained=False)
        # change the first layer to accept 32x32 images with 3 channels rather than 224x224 images
        # check the size of the input layer
        print("|| conv1 weight size: ", self.resnet.conv1.weight.size())
        print("|| fc weight size: ", self.resnet.fc.weight.size())
        self.resnet.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.resnet.fc = torch.nn.Linear(512, 10)
        self.resnet.maxpool = torch.nn.Identity()
        # change input layer to accept 32x32 images


        
        # List all layers in the resnet18 model
        for name, layer in self.resnet.named_children():
            print(f"Layer: {name} -> {layer}")
        
        #print("|| conv1 weight size: ", self.resnet.conv1.weight.size())
        #print("|| fc weight size: ", self.resnet.fc.weight.size())

    def forward(self, x):
        return self.resnet(x)
    
model = torch.load('models/resnet18_cifar77ACC.pth',  map_location=torch.device('cpu'))
model.eval()

ResNet18CIFAR(
  (resnet): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): Identity()
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
       

In [4]:
# turn the data into a tensor
test_data_tensor = torch.tensor(test_data).float()
test_labels_tensor = torch.tensor(test_labels)

# Define the number of classes in your dataset
num_classes = 10
# Initialize counters for each class
class_correct = [0] * num_classes
class_total = [0] * num_classes

print("||===================START CLASS-BY-CLASS ACCURACY=================||")

# Move tensors to GPU if available
if torch.cuda.is_available():
    test_data_tensor = test_data_tensor.cuda()
    test_labels_tensor = test_labels_tensor.cuda()

# Test the model
with torch.no_grad():
    for i in range(0, len(test_data), 1000):
        # Get the input and output
        img = test_data_tensor[i:i + 1000]
        label = test_labels_tensor[i:i + 1000]
        
        model = model.cuda()

        # Get the prediction
        outputs = model(img)
        _, predicted = torch.max(outputs, 1)

        # Update per-class counters
        for lbl, pred in zip(label, predicted):
            class_total[lbl.item()] += 1
            if lbl.item() == pred.item():
                class_correct[lbl.item()] += 1

        del img

# Print overall accuracy
overall_accuracy = sum(class_correct) / sum(class_total) * 100
print(f"|| Overall Test Accuracy: {overall_accuracy:.2f}% ||")

# Print accuracy for each class
for cls in range(num_classes):
    if class_total[cls] > 0:
        class_accuracy = class_correct[cls] / class_total[cls] * 100
        print(f"|| Accuracy for Class {cls}: {class_accuracy:.2f}% ||")
    else:
        print(f"|| No samples for Class {cls} ||")

||===================START CLASS-BY-CLASS ACCURACY=================||
|| Overall Test Accuracy: 77.34% ||
|| Accuracy for Class 0: 83.10% ||
|| Accuracy for Class 1: 88.00% ||
|| Accuracy for Class 2: 63.10% ||
|| Accuracy for Class 3: 58.30% ||
|| Accuracy for Class 4: 72.60% ||
|| Accuracy for Class 5: 66.00% ||
|| Accuracy for Class 6: 84.10% ||
|| Accuracy for Class 7: 82.50% ||
|| Accuracy for Class 8: 87.20% ||
|| Accuracy for Class 9: 88.50% ||


# UNLEARNING

In [5]:
%cd Unlearning

/dcs/21/u2110391/CS407/Machine-Unlearning-x-Continual-Learning/Unlearning


### Prepare Data and Model

In [6]:
# Classes to be Unlearnt
classes_to_unlearn = [0]

In [7]:
from collections import OrderedDict
from unlearn.scrub import train_distill, iterative_unlearn, scrub
from utils import DistillKL, AverageMeter, accuracy
from torch.utils.data import DataLoader, TensorDataset

# Convert data and labels to PyTorch tensors
train_data_tensor = torch.tensor(train_data, dtype=torch.float32).cuda()
train_labels_tensor = torch.tensor(train_labels).cuda()

test_data_tensor = torch.tensor(test_data, dtype=torch.float32).cuda()
test_labels_tensor = torch.tensor(test_labels).cuda()

# Split training data into forget and retain sets
forget_mask = torch.isin(train_labels_tensor, torch.tensor(classes_to_unlearn).cuda())
retain_mask = ~forget_mask

# Get the indices of the forget and retain subsets
forget_indices = forget_mask.nonzero(as_tuple=True)[0]
retain_indices = retain_mask.nonzero(as_tuple=True)[0]

# Create the forget and retain datasets using the indices
forget_dataset = TensorDataset(train_data_tensor[forget_indices], train_labels_tensor[forget_indices])
retain_dataset = TensorDataset(train_data_tensor[retain_indices], train_labels_tensor[retain_indices])
test_dataset = TensorDataset(test_data_tensor, test_labels_tensor)

# Create DataLoaders
data_loaders = OrderedDict(
    forget = DataLoader(forget_dataset, batch_size=64, shuffle=True),
    retain = DataLoader(retain_dataset, batch_size=64, shuffle=True),
    test =  DataLoader(test_dataset, batch_size=64, shuffle=False)
)


### DEFINE ALGORITHM HYPERPARAMETERS

In [8]:
# Define the criterion and optimizer
criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)

class Args:
    def __init__(self):
        self.unlearn_lr = 0.01         # Learning rate for unlearning
        self.momentum = 0.9
        self.weight_decay = 5e-4
        self.dataset = ''      # Change as needed
        self.num_classes = 10         # Number of classes in the dataset
        self.batch_size = 64
        self.print_freq = 10
        self.warmup = 0               # Number of warmup epochs
        self.imagenet_arch = False    # Set to True if using ImageNet architecture
        self.seed = 42       
        
        # SCRUB SPECIFIC
        self.kd_T = 1
        self.msteps = 1
        self.gamma = 10
        self.beta = 1

        # NEGRAD SPECIFIC
        self.alpha = 2


        # Add the following attributes to ensure compatibility
        self.decreasing_lr = '50,75'  # Comma-separated epochs where LR decays
        self.rewind_epoch = 0         # Epoch to rewind to; set to 0 if not using rewinding
        self.rewind_pth = ''          # Path to the rewind checkpoint
        self.gpu = 0                  # GPU ID to use; adjust as needed
        self.surgical = False         # Whether to use surgical unlearning
        self.unlearn = 'NG'           # Unlearning method, e.g., 'retrain'
        self.choice = []              # Layers to unlearn surgically; list of layer names
        self.unlearn_epochs = 10      # Number of epochs for unlearning
        self.epochs = 100   

# args = Args()

In [9]:
import copy
import unlearn

def unlearnWithArgs(data_loaders, umodel, criterion, args):
    unlearn_method = unlearn.get_unlearn_method(args.unlearn)
    if args.unlearn == 'SCRUB':
        model_s = copy.deepcopy(umodel)
        model_t = copy.deepcopy(umodel)
        module_list = nn.ModuleList([model_s, model_t])
        unlearn_method(data_loaders, module_list, criterion, args)
        umodel = module_list[0]
    else:
        unlearn_method(data_loaders, umodel, criterion, args)
    
    return umodel

In [10]:
umodel = copy.deepcopy(model)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
umodel.to(device)
umodel.train()

criterion = torch.nn.CrossEntropyLoss()

args = Args()

umodel = unlearnWithArgs(data_loaders, umodel, criterion, args)

Epoch #0, Learning rate: 0.01
len(r_loader): 704, len(f_loader): 79


Epoch: [0][9/704]	Loss 26.0853 (36.7699)	Accuracy 14.062 (11.250)	Time 1.91
Epoch: [0][19/704]	Loss 16.6894 (28.4633)	Accuracy 4.688 (11.016)	Time 0.44
Epoch: [0][29/704]	Loss 6.9493 (21.9896)	Accuracy 9.375 (11.094)	Time 0.44
Epoch: [0][39/704]	Loss 6.3166 (18.0791)	Accuracy 18.750 (11.836)	Time 0.44
Epoch: [0][49/704]	Loss 5.6424 (15.6072)	Accuracy 10.938 (12.406)	Time 0.44
Epoch: [0][59/704]	Loss 4.9331 (13.8472)	Accuracy 7.812 (13.047)	Time 0.44
Epoch: [0][69/704]	Loss 4.7695 (12.5864)	Accuracy 15.625 (13.795)	Time 0.44
Epoch: [0][79/704]	Loss 4.5330 (11.6005)	Accuracy 23.438 (15.391)	Time 0.43
Epoch: [0][89/704]	Loss 4.6587 (10.8146)	Accuracy 32.812 (16.528)	Time 0.44
Epoch: [0][99/704]	Loss 4.0813 (10.1686)	Accuracy 43.750 (18.094)	Time 0.44
Epoch: [0][109/704]	Loss 4.3673 (9.6264)	Accuracy 26.562 (19.219)	Time 0.44
Epoch: [0][119/704]	Loss 3.6716 (9.1492)	Accuracy 37.500 (20.521)	Time 0.44
Epoch: [0][129/704]	Loss 3.7057 (8.7454)	Accuracy 45.312 (21.575)	Time 0.44
Epoch: [0][139

### Evaluate Model

In [11]:
# Initialize counters for each class
from Unlearning import utils
from Unlearning.trainer.val import validate


class_correct = [0] * 10
class_total = [0] * 10

# Evaluate the unlearned model
umodel.eval()

with torch.no_grad():
    for inputs, labels in data_loaders["test"]:
        inputs = inputs.to(device).float()
        labels = labels.to(device).long()
        outputs = umodel(inputs)
        _, predicted = torch.max(outputs.data, 1)

        # Update total and correct counts for each class
        for label, prediction in zip(labels, predicted):
            class_total[label.item()] += 1
            if label.item() == prediction.item():
                class_correct[label.item()] += 1

# Print overall accuracy
overall_accuracy = 100 * sum(class_correct) / sum(class_total)
print('Overall accuracy of the unlearned model on the test data: {:.2f}%'.format(overall_accuracy))

# Print accuracy for each class
for i in range(10):
    if class_total[i] > 0:
        class_accuracy = 100 * class_correct[i] / class_total[i]
        print('Accuracy for class {}: {:.2f}%'.format(i, class_accuracy))
    else:
        print('No samples for class {}'.format(i))


for name, loader in data_loaders.items():
        utils.dataset_convert_to_test(loader.dataset, args)
        val_acc = validate(loader, umodel, criterion, args)
        print(f"{name} acc: {val_acc}")


Overall accuracy of the unlearned model on the test data: 69.51%
Accuracy for class 0: 84.80%
Accuracy for class 1: 86.20%
Accuracy for class 2: 46.20%
Accuracy for class 3: 50.20%
Accuracy for class 4: 74.90%
Accuracy for class 5: 69.10%
Accuracy for class 6: 75.30%
Accuracy for class 7: 74.20%
Accuracy for class 8: 55.60%
Accuracy for class 9: 78.60%
forget acc: 91.34
retain acc: 81.9888888888889
test acc: 69.51


### Grid Search

In [12]:
# import itertools
# import csv
# from Unlearning import utils
# from Unlearning.trainer.val import validate
 

# param_grid = {
#     'lr': [0.0001, 0.001, 0.01, 0.1],
#     'momentum': [0.8, 0.9, 0.99],
#     'weight_decay': [0.0, 1e-4, 1e-3],
#     'unlearn_epochs': [5],
# }
 

# grid_combinations = list(itertools.product(*param_grid.values()))
 
# results = []
 
# for idx, (lr, momentum, weight_decay, unlearn_epochs) in enumerate(grid_combinations):

#     print(f"Running configuration {idx + 1}/{len(grid_combinations)}: "
#     f"lr={lr}, momentum={momentum}, weight_decay={weight_decay}, unlearn_epochs={unlearn_epochs}")
 

#     umodel = copy.deepcopy(model)
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     umodel.to(device)


#     criterion = torch.nn.CrossEntropyLoss()
 

#     args = Args()
#     args.unlearn_lr = lr
#     args.momentum = momentum
#     args.weight_decay = weight_decay
#     args.unlearn_epochs = unlearn_epochs
 
#     umodel = unlearnWithArgs(data_loaders, umodel, criterion, args)
 
#     floader = data_loaders['forget']
#     utils.dataset_convert_to_test(floader.dataset, args)
#     val_acc = validate(floader, umodel, criterion, args)
#     forget_acc = val_acc

#     rloader = data_loaders['retain']
#     utils.dataset_convert_to_test(rloader.dataset, args)
#     val_acc = validate(rloader, umodel, criterion, args)
#     retain_acc = val_acc

#     tloader = data_loaders['test']
#     utils.dataset_convert_to_test(tloader.dataset, args)
#     val_acc = validate(tloader, umodel, criterion, args)
#     test_acc = val_acc
 
#     # Store results

#     results.append({

#         'lr': lr,
#         'momentum': momentum,
#         'weight_decay': weight_decay,
#         'unlearn_epochs': unlearn_epochs,
#         'retain_accuracy': retain_acc,
#         'forget_accuracy': forget_acc,
#         'test_acccuracy': test_acc,
#         'metric': retain_acc - forget_acc 

#     })
 

#     with open('grid_search_results.csv', 'w') as csvfile:
#         writer = csv.DictWriter(csvfile, fieldnames=results[0].keys())
#         writer.writeheader()
#         writer.writerows(results)
 
# print("Grid search complete. Results saved to 'grid_search_results.csv'.")
 